In [ ]:
# Import the required libraries
import pandas as pd

# Load the dataset files
members = pd.read_csv('members.csv')
songs = pd.read_csv('songs.csv')
train = pd.read_csv('train.csv')

# Explore the dataset
print("Members dataset:")
print(members.head())
print("\nSongs dataset:")
print(songs.head())
print("\nTrain dataset:")
print(train.head())


Members dataset:
                                           msno  city  bd gender  \
0  XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=     1   0    NaN   
1  UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=     1   0    NaN   
2  D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=     1   0    NaN   
3  mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=     1   0    NaN   
4  q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=     1   0    NaN   

   registered_via  registration_init_time  expiration_date  
0               7                20110820         20170920  
1               7                20150628         20170622  
2               4                20160411         20170712  
3               9                20150906         20150907  
4               4                20170126         20170613  

Songs dataset:
                                        song_id  song_length genre_ids  \
0  CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=       247640       465   
1  o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0Z

In [ ]:

!pip install scikit-surprise



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095431 sha256=841c2def170c93459014f1fd063984d2da803b2847b8583619c9c10d992bc0b0
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
# Import the required libraries
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Load the dataset files
members = pd.read_csv('members.csv')
songs = pd.read_csv('songs.csv')
train = pd.read_csv('train.csv')

# Create a Surprise Reader object
reader = Reader(rating_scale=(0, 1))

# Load the data into Surprise Dataset format
data = Dataset.load_from_df(train[['msno', 'song_id', 'target']], reader)

# Split the data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define the collaborative filtering algorithm (SVD)
algo = SVD()

# Train the algorithm on the training data
algo.fit(train_data)

# Make predictions on the test data
predictions = algo.test(test_data)

# Evaluate the performance of the model (optional)
from surprise import accuracy
accuracy.rmse(predictions)

# Get recommendations for a user
user_id = 'user123'  # replace with the actual user ID
num_recommendations = 10  # specify the number of recommendations to generate

# Retrieve the list of all songs
all_songs = songs['song_id'].unique()

# Filter out the songs already listened to by the user
listened_songs = train[train['msno'] == user_id]['song_id'].unique()
unlistened_songs = list(set(all_songs) - set(listened_songs))

# Predict ratings for unlistened songs
user_predictions = [algo.predict(user_id, song_id) for song_id in unlistened_songs]

# Sort the predictions by the estimated rating
user_predictions.sort(key=lambda x: x.est, reverse=True)

# Get the top N recommendations
top_recommendations = [pred.iid for pred in user_predictions[:num_recommendations]]

# Print the top recommendations
print("Top", num_recommendations, "recommendations for user", user_id, ":")
print(top_recommendations)


RMSE: 0.5023
Top 10 recommendations for user user123 :
['qvissY1JrwJuAO674prrBpj7qrqEvD86I0FjUi7P+CQ=', 'y9qYIofGR2f0DduejQ9dXM0penB0sKXrrW8Cl1ga3N4=', 'lsI+AkolhCnJwf3iPnbOK9znT0mZfA1i6HF08Daojxg=', 'huUbeheuJWiOxavvrOyLcKdm7ahopPp5JaSXCBNzDTg=', 'zsJQbdHU78p2g74v/cyYPGrqd3kXOxzcKDQUtbJa9YI=', 'UtF9W2EjIKQbGr/xkNHyEdg7ExGe9YnVZWrQL/QCPHs=', 'qmXuTvxf9Yg23RpTdAq2G1jN2f/fC/CMmCNrJ7tNx40=', 'QH8Ip3ePQEgosSj7vkUGOtQJPiKTkhS54SyvURhRk00=', 'Ncbo7oLyd/wjwqpC5G+zeBkZSDgEMVMm6O1OtZZElxc=', 'fJH82FxHxO9Xt6r53pEhEecUSBbIi81RCII98qaWKNE=']
